# Long Drought data check

In [1]:
# Import all python modules. You need to run this cell for the other cells to work.
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
import altair as alt
from vega_datasets import data

# for the notebook only (not for JupyterLab) run this command once per session
alt.renderers.enable('notebook')

TypeError: 'ellipsis' object is not iterable

In [ ]:
point = 'POINT(-121.46875 38.59375)'
params = {'g': point}

In [ ]:
# Cal-Adapt API
api = 'http://api.cal-adapt.org/api'

# Request header
headers = {'ContentType': 'json'}

def celsius_to_F(val):
    return val * 9/5 + 32

def kelvin_to_F(val):
    return  (val - 273.15) * 9/5 + 32

def mm_to_inches(val):
    return  val * 0.0393701

def process_daily_data(slug, params):
    # Make request
    url = api + '/series/' + slug + '/rasters/'
    response = requests.get(url, params=params)
    
    # Get data
    if response.ok:
        print('Processing:', slug)
        json = response.json()
        data = json['results'][0]

        # Multiband raster data is returned by the API as a 3D array having a shape like (233376, 1, 1)
        # Flatten the 3D array into a 1D array
        values_arr = np.array(data['image'])
        values_arr = values_arr.flatten()

        # Get total number of values -> number of days
        length = len(values_arr)

        # Get start date of timeseries
        start_date = datetime.strptime(data['event'], '%Y-%m-%d')

        # Create new pandas dataframe and map each value in list to a date index
        df = pd.DataFrame(
            values_arr,
            index=pd.date_range(start_date, freq='1D', periods=length),
            columns=['value'],
        )

        # Convert units to Fahrenheit
        units = data['units']
        if units == 'C':
            df.value = df.value.apply(lambda x: celsius_to_F(x))
        elif units == 'K':
            df.value = df.value.apply(lambda x: kelvin_to_F(x))
        elif units == 'mm':
            df.value = df.value.apply(lambda x: mm_to_inches(x))
        else:
            pass
            
        # Add slug as a column
        df['slug'] = slug  

    else:
        print('Failed to download:', slug)
            
    # Combine all the dataframes into one and return
    return df

In [ ]:
df = process_daily_data('swe_day_drought_HadGEM2-ES_rcp85_late_century', params)
df['year'] = df.index.year
df['month'] = df.index.month
df['wateryear'] = np.where(df['month'] > 9, df['year'] + 1, df['year'])
calendar_year = df.groupby(['year'])['value'].mean()
water_year = df.groupby(['wateryear'])['value'].mean()

In [ ]:
iris = data.iris()

alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth',
    color='species'
)

### Download multile climate variables

In [ ]:
variables = ['tasmax', 'tair', 'tasmin', 'pr', 'baseflow', 'et', 'runoff', 'swe']
df = []
for v in variables:
    slug = v + '_day_drought_HadGEM2-ES_rcp85_late_century'
    df_daily = process_daily_data(slug, params)
    df_daily['year'] = df_daily.index.year
    df_daily['month'] = df_daily.index.month
    df_daily['wateryear'] = np.where(df_daily['month'] > 9, df_daily['year'] + 1, df_daily['year'])
    water_year = df_daily.groupby(['wateryear'])['value'].mean()
    df.append(water_year.to_frame(v))

data = pd.concat(df, axis=1)
data

In [ ]:
data.to_csv('drought.csv')